In [168]:
def emu_list(model):
    emud = dict()
    atom_carbons = dict()
    for met, atommap in model.metabolites_info.items():
        
        if '.ex' in met:
            continue
        origatoms = atommap[0]
        if origatoms.atoms == None:
            continue
        total_atoms = origatoms.atoms[0]
        
        n_carbons = len(total_atoms)
        atom_carbons[met] = n_carbons
        
        emud[met] = ''.join([str(i) for i in range(1, n_carbons+1)])

    return (emud, atom_carbons)

In [169]:
from freeflux import Model
import random
import pandas as pd

MODEL_FILE = 'cbsimple.tsv'

model = Model('demo')
model.read_from_file(MODEL_FILE)

isim = model.simulator('inst')

EMUs = emu_list(model)[0]
num_carbons = emu_list(model)[1]
'''
isim.set_target_EMUs({
    'RUBP': '12345',
    'PP': '12345',
    'FBP': '123456',
    'F6P': '123456',
    'G6P': '123456',
    'G1P': '123456',
    'T3P': '123',
    'ADPG': '123456',
    'CO2': '1'
})
'''
isim.set_target_EMUs(EMUs)

#isim.set_target_EMUs({'rb15bp.h_12345': None})
isim.set_labeling_strategy(
    'CO2.ex', 
    labeling_pattern = ['1'], 
    percentage = [0.95], 
    purity = [1]
)

In [170]:
fluxes = {
    "T_CO2": 265.733504558412,
    "RUBISCO_CO2": 265.733504558412,
    "RUBISCO_O2": 91.8596342154520,
    "EX_2PG": 91.8596342154519,
    "GAPDHp": 623.326643331767,
    "FBAp": 901.690708753750,
    "PFPp": 132.866752279205,
    "TK3": 119.197712924601,
    "ALD": 119.197712924593,
    "SBPase": 119.197712924600,
    "TK1_f": 238.395425849188,
    "TK1_b": 0,
    "TK2_f": 119.197712924594,
    "TK2_b": 0,
    "PPI_f": 119.197712924596,
    "PPI_b": 0,
    "PRK": 357.593138773604,
    "PGIp": 556.353515250582,
    "PGMp": 942.387852878675,
    "AGP": 13.6690393545312,
    "SS": 13.6690393545314,
    "PGIp_rev": 542.684475896194,
    "PGMp_rev": 928.718813523898,
    "FBAp_rev": 768.823956475002
}

for fluxid, value in fluxes.items():
    isim.set_flux(fluxid, value)

concs = dict()
for met in model.metabolites:
    concs[met] = random.uniform(0, 10)

for concid, value in concs.items():
    isim.set_concentration(concid, value)

In [171]:
isim.set_timepoints([0, 5, 10, 20, 40])
isim.prepare(n_jobs = 1)
res = isim.simulate()

In [172]:
def mdv_to_list(mdv_obj):
    """
    Convert MDV(...) to a plain Python list, robust to different implementations.
    Works if the object is iterable, has .value, or a NumPy-like .tolist().
    """
    if hasattr(mdv_obj, "value"):
        x = mdv_obj.value
    else:
        x = mdv_obj
    try:
        return list(x)              # iterable?
    except TypeError:
        return list(x.tolist()) 
        
def build_mdv_table(res, light=None, alga=None):
    """
    Create a DataFrame where rows are timepoints and columns are concatenated
    MDV components for each EMU (e.g., E4P_0.., RUBP_0..).
    """
    # discover timepoints if not provided
    tps = [0, 5, 10, 20, 40]

    frames = []
    for emu in res.simulated_EMUs:
        label = emu.split('_')[0]  # "E4P_1234" -> "E4P"
        mdv_map = res.simulated_MDV(emu)

        # use the first timepoint to know the vector length and build column names
        first_vec = mdv_to_list(mdv_map[tps[0]])
        cols = [f"{label}_{i}" for i in range(len(first_vec))]

        data = [mdv_to_list(mdv_map[t]) for t in tps]
        df = pd.DataFrame(data, index=tps, columns=cols)
        frames.append(df)

    out = pd.concat(frames, axis=1)
    out.index.name = "LabelingTime_sec_"

    # optional metadata columns (to match your earlier table layout)
    if light is not None:
        out.insert(0, "Light", light)
    if alga is not None:
        out.insert(1 if light is not None else 0, "Alga", alga)
    out.reset_index(inplace=True)

    return out

# --- usage -----------------------------------------------------------------
# If your API exposes timepoints via keys() you can omit timepoints below.
# Otherwise, pass the ones you need explicitly:
# table = build_mdv_table(res, timepoints=[0, 5, 10, 20, 40], light="100 uE", alga="Chlamy")

table = build_mdv_table(res, light="100 uE", alga="Chlamy")
print(table)

   LabelingTime_sec_   Light    Alga        ADPG_0    ADPG_1    ADPG_2  \
0                  0  100 uE  Chlamy  9.374930e-01  0.060838  0.001645   
1                  5  100 uE  Chlamy  7.168387e-02  0.007717  0.010712   
2                 10  100 uE  Chlamy  3.461989e-05  0.000257  0.000982   
3                 20  100 uE  Chlamy  1.545932e-08  0.000002  0.000083   
4                 40  100 uE  Chlamy  1.464835e-08  0.000002  0.000081   

     ADPG_3        ADPG_4        ADPG_5        ADPG_6  ...     T3P_1  \
0  0.000024  1.924343e-07  8.325267e-10  1.500730e-12  ...  0.031417   
1  0.048607  1.267785e-01  1.782261e-01  5.562754e-01  ...  0.078462   
2  0.006355  4.018549e-02  2.270876e-01  7.250983e-01  ...  0.008299   
3  0.002097  3.004697e-02  2.302844e-01  7.374874e-01  ...  0.006977   
4  0.002079  2.996125e-02  2.302979e-01  7.375792e-01  ...  0.006977   

      T3P_2     T3P_3    x2PG_0    x2PG_1    x2PG_2   x3PGA_0   x3PGA_1  \
0  0.000340  0.000001  0.978714  0.021171  0.00

In [247]:
import re
import numpy as np
from functools import reduce
wanted_mets = ['RUBP', 'FBP', 'T3P', 'x3PGA']


def matrix(df, prefix):
    # pick only columns like ADPG_0, ADPG_1, ..., ADPG_10
    cols = [c for c in df.columns
            if isinstance(c, str) and re.fullmatch(rf'^{re.escape(prefix)}_?(\d+)$', c)]
    # sort by the numeric suffix (so _10 comes after _9)
    return df[cols].to_numpy() # (matrix, column_order)


def build_diff_mat(model, table, wanted_mets):
    
    model_S = model.get_total_stoichiometric_matrix() 
    MID_dict = {met: matrix(table, met) for met in EMUs.keys()}
    all_rxns = model_S.columns
        
    diff_mat = pd.DataFrame()

    out = {}
    
    for met in wanted_mets:

        n = num_carbons[met]
        diff_df = pd.DataFrame(0.0, index=range(n), columns=all_rxns)

        row = model_S.loc[met]
        consumers = row[row == -1].index.tolist()
        producers = row[row == 1].index.tolist()
        
        #v_self = MID_dict[met][0, :n]
        
        for rxn in consumers:
            v_self = MID_dict[met][0, :n]
            diff_df.loc[:, rxn] = -v_self

        for rxn in producers:

            rxn_col = model_S[rxn]
            print(rxn_col)
            consumed = []
            for m, coeff in rxn_col.items():
                if coeff < 0 and m != met:
                    consumed.extend([m]*int(abs(coeff)))
            print(met)
            print(consumed)

            if len(consumed) == 1:
                reactant = consumed[0]
                v_another_self = MID_dict[reactant][0, :n]                
                diff_df.loc[:, rxn] = v_another_self
            else:
                vectors = []
                for m_in in consumed:
                    if m_in not in MID_dict:
                    # if MID not available, skip (or raise)
                        continue
                    c_len = int(num_carbons[m_in])
                    vectors.append(MID_dict[m_in][0, :c_len+1])

                conv = reduce(np.convolve, vectors, np.array([1.0]))
                conv = conv[:n]
                
                diff_df.loc[:, rxn] = conv
            
        out[met] = diff_df

    return out
     
x = build_diff_mat(model, table, wanted_mets)

#for key, value in x.items():
    #print(value)


ADPG     0.0
CO2      0.0
E4P      0.0
EC2      0.0
F6P      0.0
FBP      0.0
G1P      0.0
G6P      0.0
PP      -1.0
R5P      0.0
RUBP     1.0
S7P      0.0
SBP      0.0
T3P      0.0
x2PG     0.0
x3PGA    0.0
Name: PRK, dtype: float64
RUBP
['PP']
ADPG     0.0
CO2      0.0
E4P      0.0
EC2      0.0
F6P      0.0
FBP      1.0
G1P      0.0
G6P      0.0
PP       0.0
R5P      0.0
RUBP     0.0
S7P      0.0
SBP      0.0
T3P     -2.0
x2PG     0.0
x3PGA    0.0
Name: FBAp, dtype: float64
FBP
['T3P', 'T3P']
ADPG     0.0
CO2      0.0
E4P      0.0
EC2      0.0
F6P      0.0
FBP      0.0
G1P      0.0
G6P      0.0
PP       0.0
R5P      0.0
RUBP     0.0
S7P      0.0
SBP      0.0
T3P      1.0
x2PG     0.0
x3PGA   -1.0
Name: GAPDHp, dtype: float64
T3P
['x3PGA']
ADPG     0.0
CO2      0.0
E4P      0.0
EC2      1.0
F6P      0.0
FBP      0.0
G1P      0.0
G6P      0.0
PP      -1.0
R5P      0.0
RUBP     0.0
S7P      0.0
SBP      0.0
T3P      1.0
x2PG     0.0
x3PGA    0.0
Name: TK1_b, dtype: float64
T3P
['PP']
AD

,T_CO2,RUBISCO_CO2,RUBISCO_O2,EX_2PG,GAPDHp,FBAp,PFPp,TK3,ALD,SBPase,...,PPI_f,PPI_b,PRK,PGIp,PGMp,AGP,SS,PGIp_rev,PGMp_rev,FBAp_rev
0,0.0,0.0,0.947633,0.0,-0.968242,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.051247,0.0,-0.031417,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.001109,0.0,-0.000340,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import re
import numpy as np
import pandas as pd

def matrix(df, prefix):
    cols = [c for c in df.columns
            if isinstance(c, str) and re.fullmatch(rf'^{re.escape(prefix)}_?(\d+)$', c)]
    cols = sorted(cols, key=lambda c: int(re.search(r'(\d+)$', c).group(1)))
    return df[cols].to_numpy()

def build_diff_mat(model, table, wanted_mets, tp, num_carbons, EMUs):
    """
    Returns dict[metabolite] -> DataFrame (n x #reactions) with columns filled by
    the metabolite's consumer (-) and producer (+) reactions.
    Producer source metabolite is inferred from stoichiometry: any metabolite with -1
    in that reaction column (excluding `met`). If multiple, prefer same-carbon-count.
    """
    model_S = model.get_total_stoichiometric_matrix()  # rows=metabolites, cols=reactions
    MID_dict = {met: matrix(table, met) for met in EMUs.keys()}
    all_rxns = model_S.columns

    out = {}
    for met in wanted_mets:
        n = int(num_carbons[met])
        diff_df = pd.DataFrame(0.0, index=range(n), columns=all_rxns)

        row = model_S.loc[met]
        consumers = row[row == -1].index.tolist()
        producers = row[row == 1].index.tolist()

        # vector from the metabolite's own MID (timepoint tp)
        v_self = MID_dict[met][tp, :n]

        # Fill consumers (−)
        for rxn in consumers:
            diff_df.loc[:, rxn] = -v_self

        # Fill producers (+), infer source from stoichiometry column
        for rxn in producers:
            rxn_col = model_S[rxn]
            # candidates that are consumed in this reaction (−1) and are not `met`
            candidates = [m for m in rxn_col[rxn_col == -1].index.tolist() if m != met]
            if not candidates:
                # fallback to self if nothing else is consumed (edge case)
                src_met = met
            else:
                # prefer same carbon count if available
                c_met = num_carbons.get(met)
                same_c = [m for m in candidates if num_carbons.get(m) == c_met]
                src_met = same_c[0] if same_c else candidates[0]

            v_src = MID_dict[src_met][tp, :n]
            diff_df.loc[:, rxn] = v_src

        out[met] = diff_df

    return out